In [1]:
import pandas as pd

In [6]:
dataset_1 = pd.read_pickle('../data/prepared_data/train_1_fi.pkl')
dataset_2 = pd.read_pickle('../data/prepared_data/train_2_fi.pkl')

In [7]:
dataset_1.shape, dataset_2.shape

((54000, 7), (230400, 7))